In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm # Provides progress bar for long tasks

### Listing currently existing tables in the database

In [2]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT name
    FROM sqlite_master 
    WHERE type ='table' 
    AND name NOT LIKE 'sqlite_%';
    """ 
    
    db_table_list = pd.read_sql(query, db)

display(db_table_list)

,name
0,hop_teaming
1,taxonomy
2,npidata
3,filtered_hop_teaming


### How many records do we originally have in the hop_teaming table?

**NOTE: The following cell codes runs in ~5 minutes. Set to markdown for security.**

with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(*)
    FROM hop_teaming;
    """ 
    
    db_hop_teaming_count = pd.read_sql(query, db)

display(db_hop_teaming_count)

Result: 203,330,907 records

### Filter `from_npi` to be entity type 1 and `to_npi` to be entity type 2

**NOTE: The following cell codes runs in ~5-10 minutes. Set to markdown for security.**

with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH npi_entity_type_1 AS (
        SELECT npi
        FROM npidata 
        WHERE entity_type_code = 1
    ), npi_entity_type_2 AS (
        SELECT npi
        FROM npidata 
        WHERE entity_type_code = 2
    )
    SELECT *
    FROM hop_teaming
    WHERE from_npi IN npi_entity_type_1
    AND to_npi IN npi_entity_type_2
    """
    
    filtered_hop_teaming = pd.read_sql(query, db)

display(filtered_hop_teaming.shape)
display(filtered_hop_teaming.head())

Result: 233,546

### Filter so that the `transaction_count` is >= 50 and `average_day_wait` <= 50

filtered_hop_teaming = filtered_hop_teaming[
    (filtered_hop_teaming["transaction_count"] >= 50) &
    (filtered_hop_teaming["average_day_wait"] <= 50)
]

display(filtered_hop_teaming.shape)
display(filtered_hop_teaming.head())

Final `filtered_hop_teaming` records count: 132,000

### Creating a new table called `filtered_hop_teaming`

**IMPORTANT! This loading into the database should only be run once.** If you run this multiple times, it will create duplicate entries in the database. For the security of not re-running this code by accident, the code here is converted into markdown. **If you need to rebuild the database, delete the `data/hcbb.sqlite` file and re-run this cell as code. You will also need to make sure to re-run any other related scripts that builds other tables in the database.**

with sqlite3.connect('../data/hcbb.sqlite') as db:
    filtered_hop_teaming.to_sql(
        'filtered_hop_teaming', 
        db, 
        if_exists = 'append', 
        index = False)

    # When done, print done
    print('Task done.')

### Testing

In [3]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    
    query = """
    SELECT *
    FROM filtered_hop_teaming
    LIMIT 5;
    """ 
    
    test_df = pd.read_sql(query, db)

display(test_df)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1043215882,1003028770,29,44,47.455,56.183
1,1033297429,1003028770,56,62,53.145,58.831
2,1043206329,1003028770,173,177,97.864,81.756
3,1003922881,1003028770,17,21,45.619,41.519
4,1003963976,1003028770,2535,3945,0.000,0.000


In [4]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(*) AS count_all 
    FROM filtered_hop_teaming;
    """ 
    
    test_df = pd.read_sql(query, db)

display(test_df)

,count_all
0,132000


In [5]:
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM hop_teaming
    WHERE from_npi = '1043215882'
    LIMIT 10;
    """ 
    
    test_df = pd.read_sql(query, db)

display(test_df)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1043215882,1003028770,29,44,47.455,56.183
1,1043215882,1003863580,17,18,62.500,51.261
2,1043215882,1003963976,22,27,52.852,56.225
3,1043215882,1043206329,28,31,32.290,48.225
4,1043215882,1053378505,20,24,30.042,43.446
5,1043215882,1063843951,18,53,9.887,20.953
6,1043215882,1073531042,46,51,33.549,51.752
7,1043215882,1083798136,25,36,43.167,54.234
8,1043215882,1093741464,41,65,46.585,59.003
9,1043215882,1093753303,11,12,61.333,55.082
